In [2]:
import os
from litellm import completion
from dotenv import load_dotenv
import re
load_dotenv()

API_KEY = None
try:
    # Google Colab environment
    from google.colab import userdata
    API_KEY = userdata.get('OPENROUTER_API_KEY')  # Colab secret name
except ImportError:
    # Local environment
    API_KEY = os.environ.get("OPENROUTER_API_KEY")  # Local environment variable

# response = completion(
#             model="openrouter/anthropic/claude-3.7-sonnet",
#             messages=[
#                 {"role": "user", "content": "What is the capital of France?"},
#             ]
#         )

def extract_answer_from_response(content):
    """Extracts the answer (e.g., 1, .., 7) from <ANSWER> tags."""
    match = re.search(r"<ANSWER>(.*?)</ANSWER>", content, re.IGNORECASE | re.DOTALL)
    answers = ["1", "2", "3", "4", "5", "6", "7"]
    if match and match.group(1).strip() in answers:
        return match.group(1).strip()
    # If no match, check for answers in the content
    for answer in answers:
        if answer in content:
            return answer
    return match.group(1).strip() if match else "No answer found"

In [3]:
import re



In [ ]:
import pandas as pd
import json
import os
import re
from litellm import completion
from tqdm.auto import tqdm

ring_csv_path = "results_multi/ggb_qwen-2.5-7b-instruct_ring_ensemble_260486c5_q1-90_n12.csv"
# Ensure LiteLLM can find the API key.
# The variable API_KEY should be set by the first cell of your notebook.
# If API_KEY is set, ensure it's also in os.environ for LiteLLM.
if 'API_KEY' in globals() and API_KEY:
    os.environ['OPENROUTER_API_KEY'] = API_KEY
elif not os.environ.get('OPENROUTER_API_KEY'):
    raise ValueError(
        "OPENROUTER_API_KEY not found. "
        "Please ensure it's set in Colab secrets or environment variables "
        "and the first cell of your notebook has been run successfully."
    )

# Model for classification (using the one from the example in the first cell if suitable, or adjust as needed)
CLASSIFICATION_MODEL = "openrouter/anthropic/claude-3.7-sonnet"

def classify_response_with_claude(original_question: str, agent_message_content: str):
    """
    Classifies agent's message content using Claude via LiteLLM.
    Returns a dictionary with classification results.
    """
    prompt = f"""
You are an AI assistant that classifies agent responses based on an original question.
Original Question:
---
{original_question}
---
Agent Response:
---
{agent_message_content}
---
Classify the agent's response according to the following schema. The agent's response should address the original question. Output your answer *only* as a valid JSON object.

Schema:
{{
  "is_response_off_topic": "boolean (True if the agent's rationale does not address the original question, False otherwise)",
  "off_topic_reason": "string (Provide a brief explanation if is_response_off_topic is True, otherwise null or an empty string)",
  "rationale_categories": "array of strings (Assign 1-3 categories to the rationale content. Examples: 'Utilitarianism', 'Deontology', 'Virtue Ethics', 'Rights-based', 'Consequentialism', 'Emotional Appeal', 'Refusal to Engage', 'Clarification Request', 'Agreement with others', 'Reiteration of own stance', 'Focus on Certainty/Uncertainty', 'Focus on Hypothetical Nature', 'Moral Absolutism', 'Pragmatism', 'Risk Aversion')"
}}

JSON Output:
"""
    default_error_classification = {
        "is_response_off_topic": True,
        "off_topic_reason": "Classification error",
        "rationale_categories": ["Error"]
    }
    try:
        response = completion(
            model=CLASSIFICATION_MODEL,
            messages=[
                {"role": "user", "content": prompt}
            ],
            temperature=0.0, # For deterministic classification
        )
        content = response.choices[0].message.content
        
        # Extract JSON from the response string (handles cases where LLM adds extra text)
        match = re.search(r"\{[\s\S]*\}", content)
        if match:
            json_str = match.group(0)
            try:
                classification_result = json.loads(json_str)
                # Validate essential keys
                if not all(k in classification_result for k in ["is_response_off_topic", "rationale_categories"]):
                    print(f"Warning: Claude response missing expected keys. Content: {content}")
                    return {**default_error_classification, "off_topic_reason": "Claude response format error (missing keys)"}

                # Ensure off_topic_reason is present or set to null/empty
                if "off_topic_reason" not in classification_result:
                    classification_result["off_topic_reason"] = None if not classification_result["is_response_off_topic"] else "No specific reason provided by classifier."
                
                return classification_result
            except json.JSONDecodeError as json_e:
                print(f"Warning: Could not parse JSON from Claude response. Error: {json_e}. Content: {content}")
                return {**default_error_classification, "off_topic_reason": f"Claude response JSON parsing error: {json_e}"}
        else:
            print(f"Warning: No JSON object found in Claude response. Content: {content}")
            return {**default_error_classification, "off_topic_reason": "No JSON object in Claude response"}

    except Exception as e:
        print(f"Error during Claude API call or processing: {e}")
        return {**default_error_classification, "off_topic_reason": str(e)}

# Load the CSV file.
# Ensure 'ring_csv_path' is defined in a previous cell (as per your notebook structure).
try:
    df = pd.read_csv(ring_csv_path)
    print(f"Successfully loaded CSV: {ring_csv_path}")
except NameError as ne:
    print(f"Error: {ne}")
    raise # Stop execution if CSV path is not defined
except FileNotFoundError:
    print(f"Error: The file '{ring_csv_path}' was not found. Please check the path.")
    raise
except Exception as e:
    print(f"An error occurred while loading the CSV: {e}")
    raise


classified_agent_responses_column = []

# Iterate over each row in the DataFrame with a progress bar
for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Classifying responses"):
    try:
        conversation_history_str = row.get('conversation_history', '[]')
        conversation_history = json.loads(conversation_history_str)
        
        original_question = ""
        for item in conversation_history:
            if item.get('source') == 'user' and item.get('index') == 0:
                original_question = item.get('content')
                break
        
        if not original_question:
            print(f"Warning: Row {index}: Could not find original question in 'conversation_history'. Classification may be affected.")
            # Fallback or skip if original question is crucial and missing
            # For now, we'll allow classification but it might be marked off-topic.

        agent_responses_str = row.get('agent_responses', '[]')
        agent_responses_list = json.loads(agent_responses_str)
        
        current_row_classified_responses = []
        for agent_resp_obj in agent_responses_list:
            message_content = agent_resp_obj.get('message_content', '')
            
            classification_results = None
            if not message_content:
                classification_results = {
                    "is_response_off_topic": True,
                    "off_topic_reason": "Empty message content",
                    "rationale_categories": ["Empty Content"]
                }
            elif not original_question: # If original question wasn't found for context
                 classification_results = {
                    "is_response_off_topic": True,
                    "off_topic_reason": "Original question not found for context",
                    "rationale_categories": ["Missing Context"]
                }
            else:
                classification_results = classify_response_with_claude(original_question, message_content)
            
            # Merge classification results into the agent response object
            # Create a new object to avoid modifying the original dict from the list if it's reused
            updated_agent_resp_obj = agent_resp_obj.copy()
            updated_agent_resp_obj.update(classification_results)
            current_row_classified_responses.append(updated_agent_resp_obj)
        
        classified_agent_responses_column.append(current_row_classified_responses)
        
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON for row {index}: {e}. Skipping classification for this row.")
        classified_agent_responses_column.append(f"JSON Decode Error: {e}") 
    except Exception as e:
        print(f"An unexpected error occurred for row {index}: {e}. Skipping classification for this row.")
        classified_agent_responses_column.append(f"Unexpected Error: {e}")

df['classified_agent_responses'] = classified_agent_responses_column

# Display a sample of the DataFrame with the new column
print("\nSample of DataFrame with classified responses:")
print(df[['question_id', 'classified_agent_responses']].head())

# You can save the updated DataFrame to a new CSV file if needed:
# output_csv_path = "classified_" + os.path.basename(ring_csv_path)
# df.to_csv(output_csv_path, index=False)
# print(f"\nUpdated DataFrame saved to: {output_csv_path}")

Error: 'ring_csv_path' is not defined. Please run the cell that defines this variable.


NameError: 'ring_csv_path' is not defined. Please run the cell that defines this variable.